# Hard-to-count reservations analysis

By [Ben Welsh](https://palewi.re/who-is-ben-welsh/) and [Sandhya Kambhampati]()

The analysis was conducted for the June 13, 2019, Los Angeles Times story [Headline TK](). 

It found that as the 2020 census nears, concerns about an undercount of Native Americans is gaining traction here and across the country. The finding was based, in part, on a review of public data, as well as some original analysis by The Times. 

The finding from our research:

* Nearly 600,000 Native Americans live on tribal reservations, semi-sovereign entities governed by elected indigenous leaders. On the Navajo Nation — the country’s largest reservation — 175,000 people live in a mostly rural high desert area bigger than West Virginia.

* During the 2010 count, one out of every seven Native Americans living on a reservation was missed, according to an audit by the U.S. Census Bureau. That adds up to 82,000 people overlooked and uncounted — equal to skipping the entire city of Santa Fe, New Mexico’s capital.

* The Census Bureau’s struggles in 2010 resulted in more than 80% of reservation lands being ranked among the country’s hardest-to-count areas, according to a Times analysis of estimates published by the City University of New York.

* On Navajo Nation, like many reservations, the majority of households are without a Web connection, potentially putting them at a disadvantage as the bureau moves toward collecting more data online.

Here's how we arrived at each conclusion.

### Import Python tools

The tools we used to do our work.

In [75]:
import warnings
import pandas as pd
import geopandas as gpd

In [76]:
warnings.simplefilter("ignore")

Utilities to download Census data from the Data Desk's [census-data-downloader](https://github.com/datadesk/census-data-downloader/)

In [13]:
read_acs = lambda x: pd.read_csv(
    f"https://raw.githubusercontent.com/datadesk/census-data-downloader/ba0ca0fcf30ebcaf7a42d2babbccb1b057312e3f/data/processed/acs5_2017_{x}.csv",
    dtype={"geoid": str}
)

### The Native American population

After the 2010 count, the Census Bureau conducted an internal audit known as the "Coverage Measurement Analysis." It concluded that there are roughly 600,000 Native Americans living on a federal reservations. This number was confirmed by a press office at the bureau.

![](img/na-population.gif)

Navajo Nation's total population, roughly 175,000, is available from the bureau's American Community Survey.

In [14]:
raw_homeland_population = read_acs("population_aiannhhomeland")

In [15]:
raw_homeland_population[raw_homeland_population.name.str.contains("Navajo Nation")]

,geoid,name,universe,american indian area/alaska native area/hawaiian home land
175,2430,Navajo Nation Reservation and Off-Reservation ...,175005.0,2430


### The Native American undercount

The "Coverage Measurement Analysis" also estimated the the undercount of Native Americans on reservations. It found a statistically significant net 5% undercount, which includes people counted twice, people imputed entirely by statistics and people missed entirely. The total number of people missed, known as the omission rate, was 13.7%. This number was confirmed by the press office at the bureau.

![](img/na-undercount.gif)

That adds up to 82,000 people overlooked and uncounted — equal to skipping the entire city of Santa Fe, New Mexico’s capital.

In [16]:
600000*0.137

82200.0

In [17]:
raw_place_population = read_acs("population_places")

In [18]:
raw_place_population[raw_place_population.universe < 83000].sort_values(
    "universe",
    ascending=False
).head(5)[[
    'name',
    'universe'
]]

,name,universe
16750,"Santa Fe city, New Mexico",82980.0
8944,"Sioux City city, Iowa",82568.0
4568,"Largo city, Florida",82433.0
4739,"Palm Coast city, Florida",82356.0
25428,"Longview city, Texas",82219.0


### The hard-to-count areas

The Census Bureau’s struggles in 2010 resulted in more than 80% of reservation lands being ranked among the country’s hardest-to-count areas, according to a Times analysis of estimates published by the City University of New York.

#### Hard to count tracts

Import every Census tract with its hard-to-count status according to the [Center for Urban Research at the City University of New York Graduate Center](https://www.censushardtocountmaps2020.us/).

In [19]:
tracts = gpd.read_file("data/analysis/hard-to-count-tracts.geojson")

In [20]:
tracts.head()

,geoid,state,county,pop,mrr,mrr_htc,ue,htc,geometry
0,01005950300,Alabama,Barbour County,1813,77.9,0,0,0,POLYGON ((-85.52744143500958 31.86650898512998...
1,01005950900,Alabama,Barbour County,3888,74.6,0,0,0,POLYGON ((-85.16412632985951 31.83060098962552...
2,01005950800,Alabama,Barbour County,2157,83.5,0,0,0,POLYGON ((-85.14872232914963 31.90934100751823...
3,01005950700,Alabama,Barbour County,1775,79.6,0,0,0,POLYGON ((-85.14578832747586 31.89149700368442...
4,01005950600,Alabama,Barbour County,2120,79.4,0,0,0,"POLYGON ((-85.14572732739356 31.8901120033811,..."


How many Census tracts are there total?

In [21]:
len(tracts)

72845

Filter down to just the hard to count tracts

In [22]:
htc = tracts[tracts.htc >= 1]

How many hard to county tracts are there?

In [23]:
len(htc)

14800

Write that out to a SHP

In [24]:
htc.to_file("./data/htc-tracts.shp")

#### Tribal reservations

Read in tribal homelands from the [U.S. Census Bureau's TIGER/Line Shapefiles service](https://www.census.gov/cgi-bin/geo/shapefiles/index.php).

In [25]:
raw_homelands = gpd.read_file("data/tiger/tl_2018_us_aiannh/tl_2018_us_aiannh.shp")

Clean that up.

In [27]:
trimmed_homelands = raw_homelands[['AIANNHCE', 'NAMELSAD', 'geometry']]

In [28]:
prepped_homelands = trimmed_homelands.rename(columns={
    "AIANNHCE": "geoid",
    "NAMELSAD": "name",
})

The homelands come in a variety of types. Only a subset are the federally-recognized resevations we want to study.

In [30]:
def get_geotype(geoid):
    """
    Accepts the GEOID of a American Indian, Alaska Native or Native Hawaiian homeland.
    
    Returns which type of homeland it is.
    
    They are defined on pages 47 and 48 of this Census Bureau document:
    
        https://www.census.gov/prod/cen2010/doc/pl94-171.pdf
    """
    val = int(geoid)
    if val < 5000:
        return 'Federal reservation or trust land'
    elif val < 5500:
        return 'Hawaiian home land'
    elif val < 6000:
        return 'Oklahoma tribal statistical area'
    elif val < 8000:
        return 'Alaska native village statistical area'
    elif val < 9000:
        return 'Tribal-designated statistical area'
    elif val < 9500:
        return 'State reservation'
    elif val < 9999:
        return 'State-designated tribal statistical area'

In [32]:
prepped_homelands['geotype'] = prepped_homelands.geoid.apply(get_geotype)

In [33]:
prepped_homelands.geotype.value_counts(dropna=False)

Federal reservation or trust land           477
Alaska native village statistical area      220
Hawaiian home land                           75
State-designated tribal statistical area     30
Oklahoma tribal statistical area             29
State reservation                            10
Tribal-designated statistical area            4
Name: geotype, dtype: int64

Filter down to just the reservations.

In [35]:
reservations = prepped_homelands[prepped_homelands.geotype == 'Federal reservation or trust land']

Dissolve reservations and trust lands into single shape using their common unique identifier.

In [36]:
reservations_deduped = reservations.dissolve(by='geoid').reset_index()

Run the count again.

In [37]:
reservations_deduped.geotype.value_counts(dropna=False)

Federal reservation or trust land    327
Name: geotype, dtype: int64

Calculate the total area of each reservation.

In [38]:
reservations_deduped['area'] = reservations_deduped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

Show the biggest ones.

#### Geospatial analysis

Reproject the reservations to match the tracts.

In [40]:
reservations_reprojected = reservations_deduped.to_crs(htc.crs)

Return the areas that intersect between the reservations and the hard-to-count tracts.

In [41]:
intersection = gpd.overlay(
    reservations_reprojected,
    htc,
    how='intersection'
)

How many intersections are there?

In [42]:
len(intersection)

553

Dissolve the intersecting tracts using each reservations unique identifier.

In [43]:
dissolved = intersection.dissolve(by='geoid_1')

How many reservations intersect with a hard-to-count tract?

In [44]:
len(dissolved)

170

Clean up the dataframe.

In [45]:
dissolved_trimmed = dissolved.reset_index()[[
    'geoid_1',
    'name',
    'geometry'
]]

In [46]:
dissolved_prepped = dissolved_trimmed.rename(columns={
    "geoid_1": "geoid",
})

Calculate the total area of each intersection.

In [47]:
dissolved_prepped['area'] = dissolved_prepped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

Merge intersections with the reservations.

In [48]:
dissolved_merged = dissolved_prepped.merge(
    reservations_deduped[['geoid', 'area']],
    on="geoid",
    suffixes=["_htc", "_total"]
)

Calculate the percent of each reservation's area that intersects with a hard-to-count tract.

In [49]:
dissolved_merged['pct_htc'] = dissolved_merged.area_htc / dissolved_merged.area_total

Show the largest percentages.

In [50]:
dissolved_merged.sort_values("area_htc", ascending=False).head(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
88,2430,Navajo Nation Reservation,"(POLYGON ((-108.344659 34.711661, -108.3446588...",62315.774112,62565.921391,0.996002
151,4390,Uintah and Ouray Reservation,POLYGON ((-110.0241408869228 39.46170078192419...,14963.114044,17674.267493,0.846604
144,4200,Tohono O'odham Nation Reservation,"(POLYGON ((-112.719808 32.967355, -112.719534 ...",11534.902203,11534.986075,0.999993
18,0605,Cheyenne River Reservation,POLYGON ((-101.8710226103383 44.53181420775375...,11424.555187,11445.301110,0.998187
139,3970,Standing Rock Reservation,POLYGON ((-101.4702072145578 45.47181753760831...,9486.074003,9486.191063,0.999988


Show the smallest percentages.

In [51]:
dissolved_merged.sort_values("area_htc", ascending=False).tail(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
99,2715,Pauma and Yuima Reservation,(POLYGON ((-116.9321159667102 33.3464051059000...,0.007937,24.252290,3.272689e-04
67,1850,La Jolla Reservation,(POLYGON ((-116.8410570065867 33.2559419054459...,0.002918,34.957758,8.348507e-05
15,0495,Capitan Grande Reservation,POLYGON ((-116.6811140223018 32.88451265870786...,0.002156,64.430007,3.346085e-05
148,4315,Tunica-Biloxi Reservation,(POLYGON ((-92.0626268708872 31.10577978474263...,0.000030,3.149779,9.520608e-06
96,2635,Pala Reservation,"POLYGON ((-117.000974922776 33.38656981010941,...",0.000006,52.706195,1.098324e-07


Write out the result to a GeoJSON file.

In [52]:
dissolved_merged.to_file("data/analysis/htc-reservations.geojson", driver="GeoJSON")

How much total reservation land is hard to count?

In [53]:
dissolved_merged.area_htc.sum()

239727.28813155653

How much total reservation land is there overall?

In [54]:
reservations_deduped['area'].sum()

295020.57328184653

What percentage of reservation land is hard to count?

In [55]:
dissolved_merged.area_htc.sum() / reservations_deduped['area'].sum()

0.8125782058681521

### Internet on reservations

On Navajo Nation, like many reservations, the majority of households are without a Web connection, potentially putting them at a disadvantage as the bureau moves toward collecting more data online.

In [57]:
raw_homeland_internet = read_acs("internet_aiannhhomeland")

In [62]:
raw_homeland_internet['percent_no_internet'] = raw_homeland_internet.total_no_internet / raw_homeland_internet.universe

In [66]:
trimmed_homeland_internet = raw_homeland_internet[[
    'geoid',
    'name',
    'total_no_internet',
    'universe',
    'percent_no_internet'
]]

In [77]:
trimmed_homeland_internet['geotype'] = trimmed_homeland_internet.geoid.apply(get_geotype)

In [69]:
reservations_internet = trimmed_homeland_internet[trimmed_homeland_internet.geotype == 'Federal reservation or trust land']

In [71]:
reservations_internet.sort_values(
    "universe",
    ascending=False
).head()

,geoid,name,total_no_internet,universe,percent_no_internet,geotype
175,2430,Navajo Nation Reservation and Off-Reservation ...,35016.0,45972.0,0.761681,Federal reservation or trust land
172,2595,"Osage Reservation, OK",6029.0,18314.0,0.329202,Federal reservation or trust land
210,3000,Puyallup Reservation and Off-Reservation Trust...,2966.0,17949.0,0.165246,Federal reservation or trust land
30,0020,Agua Caliente Indian Reservation and Off-Reser...,2218.0,13777.0,0.160993,Federal reservation or trust land
63,1110,"Flathead Reservation, MT",3787.0,11550.0,0.327879,Federal reservation or trust land


In [74]:
len(reservations_internet)

326

In [73]:
len(reservations_internet[reservations_internet.percent_no_internet >= .5])

78